In [1]:
import os

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas import Series, DataFrame

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV, cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient

In [2]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://truenas.local:9000"
os.environ["MLFLOW_TRACKING_URI"] = "http://192.168.1.14:5000"

EXPERIMENT_NAME = "Weather Forecast Model Experiment"

In [3]:
client = MlflowClient()

In [5]:
reg_model = client.get_registered_model("KrasnodarWeatherForecastModel")

In [10]:
model_uri = "models:/KrasnodarWeatherForecastModel/production"
loaded_model = mlflow.pyfunc.load_model(model_uri)

2023/07/11 14:46:18 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.2, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [11]:
loaded_model.metadata

In [ ]:
data = {}